# Recommender Systems
Developing recommender systems is one of the most popular applications of data science in consumer apps.  
Suggest a recommender system that you believe we could use at Fetch Rewards
Describe the steps you would follow to develop the system
Explain how you would define success
[Bonus] Demonstrate one or more of your suggested approaches in any tool, on any data set of your choosing, from any domain, real or made-up


# Recommender toy data set obtained from 

https://grouplens.org/datasets/movielens/latest/


In [1]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity


In [2]:
movies_df = pd.read_csv('Datasets/movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings_df = pd.read_csv('Datasets/ratings.csv')
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
merged_df = pd.merge(movies_df,ratings_df, on='movieId')
merged_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [5]:
merged_df.drop(columns=['genres','timestamp'], inplace=True)
merged_df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [6]:
# create pivot table to feed into pairwise metrics
pivot = pd.pivot_table(merged_df, index='title', columns='userId', values='rating')
pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# create sparse pivot table to make data set smaller pairwise distance and cosine similiarity need NAN values = 0
sparse_pivot = sparse.csr_matrix(pivot.fillna(0))

In [8]:
# cosine metric creates scale from 0 to 1
recommender = pairwise_distances(sparse_pivot,metric='cosine')

In [9]:
recommender_df = pd.DataFrame(recommender, columns=pivot.index, index=pivot.index)
recommender_df.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,0.858347,1.0,...,1.0,0.657945,0.456695,0.292893,1.0,1.0,0.860569,0.672673,1.0,1.0
'Hellboy': The Seeds of Creation (2004),1.0,0.000000,0.292893,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0
'Round Midnight (1986),1.0,0.292893,0.000000,1.000000,1.000000,1.0,0.823223,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0
'Salem's Lot (2004),1.0,1.000000,1.000000,0.000000,0.142507,1.0,1.000000,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0
'Til There Was You (1997),1.0,1.000000,1.000000,0.142507,0.000000,1.0,1.000000,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0


In [10]:
# Finding exact name in database of Milo and Otis
movies_df.loc[movies_df['title'].str.contains('Milo')]


,movieId,title,genres
2138,2846,"Adventures of Milo and Otis, The (Koneko monog...",Adventure|Children|Comedy|Drama
8191,103210,Fullmetal Alchemist: The Sacred Star of Milos ...,Action|Adventure|Animation
8270,105325,Bad Milo (Bad Milo!) (2013),Comedy|Horror


In [11]:
# pull out title only to feed into recommender_df
movies_df.loc[2138]['title']

'Adventures of Milo and Otis, The (Koneko monogatari) (1986)'

# Top ten movie recommendations for Adventures of Milo and Otis based on distance from Milo and Otis and their distance values.


In [12]:
# top ten movie recommendations for Adventures of Milo and Otis based on distance from Milo and Otis
recommender_df[movies_df.loc[2138]['title']].sort_values()[1:11]

title
Virus (1999)                                             0.285410
Adventures of Huck Finn, The (1993)                      0.358497
At Close Range (1986)                                    0.381158
Leprechaun (1993)                                        0.411495
Lost & Found (1999)                                      0.414783
Little Giants (1994)                                     0.421994
Blank Check (1994)                                       0.424658
Superstar (1999)                                         0.433330
Arthur (1981)                                            0.435831
King Kong vs. Godzilla (Kingukongu tai Gojira) (1962)    0.439551
Name: Adventures of Milo and Otis, The (Koneko monogatari) (1986), dtype: float64

---

#### Bonus Recommender System

##### My Dog CC recommends that I feed him more bacon

<img src="cc.jpg" alt="drawing" width="400"/>

